In [14]:
import json
import logging
import re
import folium

from django.contrib.gis.geos.polygon import Polygon

from madmex.api.remote import UsgsApi, EspaApi
from madmex.management.base import AntaresBaseCommand
from madmex.models import Country, Footprint, Region, Order


In [15]:
usgs_client = UsgsApi()

In [16]:
usgs_client.login()

https://earthexplorer.usgs.gov/inventory/json/v/stable/login
Successfully logged in to earth explorer USGS api.


True

In [69]:
start_date = '2001-01-01' 
end_date = '2001-02-31'
landsat = 8
shape_name = 39039
cloud_cover = 10

In [70]:
espa_client = EspaApi()

In [71]:
shape_object = Footprint.objects.get(name=shape_name)

In [72]:
tile = json.loads(shape_object.the_geom.geojson)
m = folium.Map(location=[24, -102],
               tiles='OpenStreetMap',
               zoom_start=5.5)
folium.GeoJson(tile).add_to(m)
m

In [73]:
extent = shape_object.the_geom.extent

In [74]:
# Landsat 5
collection_usgs = 'LANDSAT_TM_C1'
collection_espa = 'tm5_collection'
collection_regex = '^lt05_{1}\\w{4}_{1}[0-9]{6}_{1}[0-9]{8}_{1}[0-9]{8}_{1}[0-9]{2}_{1}\\w{2}$'

In [75]:
data = usgs_client.search(extent, collection_usgs, start_date=start_date, end_date=end_date, max_cloud_cover=cloud_cover).get('data')

https://earthexplorer.usgs.gov/inventory/json/v/stable/search


In [76]:
data

{'numberReturned': 7,
 'totalHits': 7,
 'firstRecord': 1,
 'lastRecord': 7,
 'nextRecord': 7,
 'results': [{'acquisitionDate': '2001-01-06',
   'startTime': '2001-01-06',
   'endTime': '2001-01-06',
   'lowerLeftCoordinate': {'latitude': 31.10193, 'longitude': -117.22596},
   'upperLeftCoordinate': {'latitude': 32.68613, 'longitude': -116.81644},
   'upperRightCoordinate': {'latitude': 32.40376, 'longitude': -114.87537},
   'lowerRightCoordinate': {'latitude': 30.82421, 'longitude': -115.31654},
   'spatialFootprint': None,
   'sceneBounds': '-117.22596,30.82421,-114.87537,32.68613',
   'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_tm_c1/2001/039/038/LT05_L1TP_039038_20010106_20160918_01_T1.jpg',
   'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_TM_C1&ordered=LT50390382001006XXX02&node=INVSVC',
   'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/LANDSAT_TM_C1/LT50390382001006XXX02/INVSVC/',
   'entityId': 'LT5039038200